<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: 7. Linear systems
## LU decomposition
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we implement **LU decomposition** as a method to solve 
a system of linear equations.

In [1]:
import numpy as np
import numpy.linalg

----
## LU decomposition

We need to two steps for the method.

1. The **first step** is the decomposition of the original matrix ${\bf A}$ into lower ${\bf L}$ and
upper ${\bf U}$ triangular matrix, using the LU decomposition algorithm: 

Examine in `python`:

In [2]:
def lin_lu_decompose(a):
    '''
    #----------------------------------------------------------------------
    # subroutine decomposes matrix A 
    # into lower L and upper U triangular matrices, using LU decompostion
    # Input:
    # a(n,n)  - coefficient matrix
    # Output:
    # l(n,n)  - lower triangular matrix
    # u(n,n)  - upper triangular matrix
    # (c) Georg Kaufmann
    #----------------------------------------------------------------------
    '''
    n = a.shape[0]
    l = np.zeros([n,n])
    u = np.zeros([n,n])
    for j in np.arange(1,n+1): # 1,n
        l[j-1][j-1] = 1.
        u[1-1][j-1] = a[1-1][j-1]
        l[j-1][1-1] = a[j-1][1-1] / u[1-1][1-1]
    for i in np.arange(2,n+1): # 2,n
        sum = 0.
        for k in np.arange(1,i): # 1,i-1
            sum = sum + l[i-1][k-1]*u[k-1][i-1]
        u[i-1][i-1] = a[i-1][i-1] - sum
        for j in np.arange(i+1,n+1): # i+1,n
            sum = 0.
            for k in np.arange(1,i): # 1,i-1
                sum = sum + l[i-1][k-1]*u[k-1][j-1]
            u[i-1][j-1] = (a[i-1][j-1] -  sum) / l[i-1][i-1]
            sum = 0.
            for k in np.arange(1,i): # 1,i-1
                sum = sum + l[j-1][k-1]*u[k-1][i-1]
            l[j-1][i-1] = (a[j-1][i-1] - sum) / u[i-1][i-1]
    return l,u

Test with matrix **A**:
$$
{\bf A} = \left[
\begin{array}{ccc}
2&1&-1 \\ 1&3&1 \\ -1&1&4
\end{array}
\right]
$$

In [3]:
a=np.array([[2.,1.,-1.],[1.,3.,1.],[-1.,1.,4.]])

a2=np.copy(a)
print('A_ij:\n',a,a.ndim,a.shape)
l,u = lin_lu_decompose(a2)
print('L_ij:\n',l,l.ndim,l.shape)
print('U_ij:\n',u,u.ndim,u.shape)

A_ij:
 [[ 2.  1. -1.]
 [ 1.  3.  1.]
 [-1.  1.  4.]] 2 (3, 3)
L_ij:
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  0.6  1. ]] 2 (3, 3)
U_ij:
 [[ 2.   1.  -1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.   2.6]] 2 (3, 3)


2. The **second step** is the decomposition of the original matrix:
Solve for ${\bf y}$ with forward substitution: 
$$
 y_i = \frac{1}{l_{ii}} \left( b_i - \sum\limits_{j=1}^{n-1} l_{ij} y_j \right), i=1,n
$$

Solve for ${\bf x}$ through backward substitution:
$$
 x_i = \frac{1}{u_{ii}} \left( y_i - \sum\limits_{j=i+1}^{n} u_{ij} x_j \right), i=n,1,-1
$$
 

Examine in `python`:

In [4]:
def lin_lu_solve(l,u,b):
    '''
    #----------------------------------------------------------------------
    # subroutine solves the system of linear equations
    # a(n,n)*x(n) = b(n)
    # using the lower and upper triangular matrices L and U
    # obtained from LU decomposition
    # Input:
    # a(n,n)  - coefficient matrix
    # b(n)    - rhs vector
    # Output:
    # x(n)    - solution vector
    # (c) Georg Kaufmann
    #----------------------------------------------------------------------
    '''
    n = len(b)
    # solve decomposed system Ly=b with forward substitution
    for i in np.arange(1,n+1): # 1,n
        sum = 0.
        for j in np.arange(1,i): # 1,i-1
            sum = sum + l[i-1][j-1] * b[j-1]
        b[i-1] = (b[i-1] - sum) / l[i-1][i-1]
    # solve decomposed system Ux=y with backward substitution
    x = np.zeros([n])
    for i in np.arange(n-1,-1,step=-1): # n,1,-1
        sum = 0.0
        for j in np.arange(i+1,n): # i+1,n
            sum = sum + u[i][j] * x[j]
        x[i] = (b[i] - sum) / u[i][i]
    return x

## Examples

We test the **LU decomposition method** with two examples:

1. ${\bf A} {\bf x} = {\bf b}$

with 
$$
{\bf A} = \left[
\begin{array}{ccc}
2&1&-1 \\ 1&3&1 \\ -1&1&4
\end{array}
\right];
{\bf b} = \left[
\begin{array}{c} 4 \\ 3 \\ 4 \end{array}
\right]
$$
The solution vector is ${\bf x} \simeq (3.46,-0.85,2.08)$.

In [5]:
a=np.array([[2.,1.,-1.],[1.,3.,1.],[-1.,1.,4.]])
b=np.array([4.,3.,4.])

a2=np.copy(a); b2=np.copy(b)
print('A_ij:\n',a,a.ndim,a.shape)
print('b_j: \n',b,b.ndim,b.shape)
l,u=lin_lu_decompose(a2)
x = lin_lu_solve(l,u,b2)
print('x_i: \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

# check against numpy solution
a2=np.copy(a); b2=np.copy(b)
x = np.linalg.solve(a2, b2)
print('x_i: \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

A_ij:
 [[ 2.  1. -1.]
 [ 1.  3.  1.]
 [-1.  1.  4.]] 2 (3, 3)
b_j: 
 [4. 3. 4.] 1 (3,)
x_i: 
 [ 3.46153846 -0.84615385  2.07692308] 1 (3,)
True
x_i: 
 [ 3.46153846 -0.84615385  2.07692308] 1 (3,)
True


2. ${\bf A} {\bf x} = {\bf b}$

with 
$$
{\bf A} = \left[
\begin{array}{cccc}
  1 &  1 &  0 &  3 \\
  2 &  1 & -1 &  1 \\
  3 & -1 & -1 &  2 \\
 -1 &  2 &  3 & -1 
\end{array} 
\right];
{\bf b} = \left[
\begin{array}{c} 4 \\ 1 \\ -3 \\ 4 \end{array}
\right]
$$
The solution vector is ${\bf x} = (−1,2,0,1)$.

In [6]:
a=np.array([[1,1,0,3],[2,1,-1,1],[3,-1,-1,2],[-1,2,3,-1]])
b=np.array([4,1,-3,4])

a2=np.copy(a); b2=np.copy(b)
print('A_ij:\n',a,a.ndim,a.shape)
print('b_j: \n',b,b.ndim,b.shape)
a2=np.copy(a); b2=np.copy(b)
l,u=lin_lu_decompose(a2)
x = lin_lu_solve(l,u,b2)
print('x_i: \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

# check against numpy solution
a2=np.copy(a); b2=np.copy(b)
x = np.linalg.solve(a2, b2)
print('x_i: \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

A_ij:
 [[ 1  1  0  3]
 [ 2  1 -1  1]
 [ 3 -1 -1  2]
 [-1  2  3 -1]] 2 (4, 4)
b_j: 
 [ 4  1 -3  4] 1 (4,)
x_i: 
 [-1.  2.  0.  1.] 1 (4,)
True
x_i: 
 [-1.00000000e+00  2.00000000e+00 -7.40148683e-17  1.00000000e+00] 1 (4,)
True


[next>](Numerics_lab07_det_inverse.ipynb)